In [ ]:
import os
import time
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from numpy.linalg import svd, norm
from memory_profiler import memory_usage

# --- File paths ---
base_path = r'G:\My Drive\NUS\NUS Y6S1\ME5311\PROJECT_2420_ME5311'
slp_path = os.path.join(base_path, 'slp.nc')
t2m_path = os.path.join(base_path, 't2m.nc')

# --- Load datasets ---
ds_slp = xr.open_dataset(slp_path)
ds_t2m = xr.open_dataset(t2m_path)

slp = ds_slp['msl'].values
t2m = ds_t2m['t2m'].values
timestamps = ds_slp['time'].values
lats = ds_slp['latitude'].values
longs = ds_slp['longitude'].values

# --- Reshape and center SLP ---
n_time, n_lat, n_lon = slp.shape
A_slp = slp.reshape(n_time, -1).T
A_mean_slp = A_slp.mean(axis=1, keepdims=True)
A_centered_slp = A_slp - A_mean_slp

In [ ]:
# --- Economy SVD Evaluation on SLP (full_matrices=False) ---

def perform_economy_svd(A_centered):
    def svd_task():
        global U_econ, S_econ, VT_econ
        U_econ, S_econ, VT_econ = svd(A_centered, full_matrices=False)

    start_time = time.time()
    mem_peak = memory_usage(svd_task, max_usage=True)
    elapsed = time.time() - start_time
    return elapsed, mem_peak

print("Performing economy SVD on SLP...")
elapsed_econ, peak_mem_econ = perform_economy_svd(A_centered_slp)

# --- Reconstruct and compute accuracy ---
S_mat_econ = np.diag(S_econ)
A_reconstructed_econ = U_econ @ S_mat_econ @ VT_econ + A_mean_slp
reconstruction_error_econ = norm(A_slp - A_reconstructed_econ) / norm(A_slp)

# --- Noise robustness ---
np.random.seed(0)
noise = np.random.normal(scale=0.01, size=A_centered_slp.shape)
A_noisy = A_centered_slp + noise
U_noisy_econ, S_noisy_econ, VT_noisy_econ = svd(A_noisy, full_matrices=False)
A_reconstructed_noisy_econ = U_noisy_econ @ np.diag(S_noisy_econ) @ VT_noisy_econ + A_mean_slp
noise_error_econ = norm(A_slp - A_reconstructed_noisy_econ) / norm(A_slp)

# --- Print evaluation results ---
print("\n===== Economy SVD Results for SLP =====")
print(f"A shape: {A_slp.shape}")
print(f"U shape: {U_econ.shape}, S shape: {S_econ.shape}, VT shape: {VT_econ.shape}")
print(f"Runtime: {elapsed_econ:.2f} seconds")
print(f"Peak memory usage: {peak_mem_econ:.2f} MiB")
print(f"Reconstruction error (Frobenius norm): {reconstruction_error_econ:.6e}")
print(f"Noise robustness (error with Gaussian noise): {noise_error_econ:.6e}")


In [ ]:
# --- Report results ---
print("\n===== Economy SVD Results for SLP =====")
print(f"Runtime: {elapsed_econ:.2f} seconds")
print(f"Peak memory usage: {peak_mem_econ:.2f} MiB")
print(f"Reconstruction error (Frobenius norm): {reconstruction_error_econ:.6e}")
print(f"Noise robustness (error with Gaussian noise): {noise_error_econ:.6e}")

# --- Optional: Save results to file for later comparison ---
results = {
    "method": "Economy SVD",
    "runtime": float(elapsed_econ),
    "memory_usage": float(peak_mem_econ),
    "reconstruction_error": float(reconstruction_error_econ),
    "noise_robustness": float(noise_error_econ),
    "top_singular_values": [float(sv) for sv in S_econ[:10]]  # Convert numpy float32 to Python float
}

# Save as JSON (optional)
import json
with open("Economy_svd_results.json", "w") as f:
    json.dump(results, f, indent=4)

# --- Optional: Plot singular value decay ---
plt.figure(figsize=(10, 6))
plt.semilogy(range(1, 101), S_econ[:100], 'o-')
plt.title('Singular Value Decay (Top 100)')
plt.xlabel('Index')
plt.ylabel('Singular Value (log scale)')
plt.grid(True)
plt.savefig('full_svd_singular_values.png', dpi=300)
plt.show()